# Reactions
This notebook contains all the steps followed in order to reconcile the existing metabolic reconstruction in CHO cells. It si divided into two parts: **1. Network Reconstruction** and **2. Identification of duplicated reactions**

[1. Network Reconstruction](#reconstruction) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.1 Datasets generation and merge**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.2 Normalization of the data** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.3 Group all the data into a unified dataset** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.4 Addition of the Recon3D CHO ortholog GPRs into the reconstruction** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.5 Divide the dataset in two to facilitate reading and curation in Google Sheets** <br>

[2. Identification of Duplicated Reactions](#duplicated) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.1 Generate the model and identify duplicated reactions**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.2 Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above** <br>

[3. Divide into compartments](#compartments) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.1 Generate the model and identify duplicated reactions**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.2 Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above** <br>

## 1. Network Reconstruction <a id='reconstruction'></a>
These include the generation of datasets (1), normalization of the data (2), merging all four reconstructions into a unified dataset (3), compiling the generated reconstruction into a cobra model in order to identify duplicated reactions (4), fixing duplicated reactions using BiGG ID annotation (5), adding Recon3D GPR information (6), and finally dividing the dataset into two different datasets that will be further curated in Google Sheets (7).

In [ ]:
# Libraries import
import pandas as pd
import numpy as np

### 1.1 Datasets generation and merge
Dataset generation from previous reconstructions (CHO_DG44, iCHO1766, iCHO2101, iCHO2291)

In [ ]:
#Read excel files and create the dfs

#Camels CHO_DG44 metabolic reconstruction
camel_df = pd.read_excel('../Data/Reconciliation/datasets/CHO_DG44.xlsx', header = 1)

#Hefzi's iCHO1766 metabolic reconstruction
hefzi_df = pd.read_excel('../Data/Reconciliation/datasets/hefzi_final.xlsx')

#Foudaliha's iCHO2101 metabolic reconstruction
fouladiha_df = pd.read_excel('../Data/Reconciliation/datasets/iCHO2101.xlsx', 'Supplementary Table 10', header = 1)

#Yeo's iCHO2291 metabolic reconstruction
iCHO2291 = pd.read_excel('../Data/Reconciliation/datasets/iCHO2291_final.xlsx')

### 1.2 Normalization of the data
All dataset are normalized into the same shape and format and then combined into one big dataset

In [ ]:
#Standarization of the columns names
camel_df.rename(columns = {'Reaction ID':'Reaction', 'Initial reaction in model':'Reaction Formula', 'Reaction name':'Reaction Name', 'Justification':'Curation Notes'}, inplace = True)
fouladiha_df.rename(columns = {'Abbreviation':'Reaction', 'Description':'Reaction Name', 'Reaction':'Reaction Formula', 'GPR':'GPR_fou'}, inplace = True)

# Addition of tag columns for organizational purposes
camel_df.insert(loc=0, column='cam', value='X')
camel_df.insert(loc=1, column='hef', value=np.nan)
camel_df.insert(loc=2, column='fou', value=np.nan)
camel_df.insert(loc=3, column='yeo', value=np.nan)

hefzi_df.insert(loc=0, column='cam', value=np.nan)
hefzi_df.insert(loc=1, column='hef', value='X')
hefzi_df.insert(loc=2, column='fou', value=np.nan)
hefzi_df.insert(loc=3, column='yeo', value=np.nan)

fouladiha_df.insert(loc=0, column='cam', value=np.nan)
fouladiha_df.insert(loc=1, column='hef', value=np.nan)
fouladiha_df.insert(loc=2, column='fou', value='X')
fouladiha_df.insert(loc=3, column='yeo', value=np.nan)

iCHO2291.insert(loc=0, column='cam', value=np.nan)
iCHO2291.insert(loc=1, column='hef', value=np.nan)
iCHO2291.insert(loc=2, column='fou', value=np.nan)
iCHO2291.insert(loc=3, column='yeo', value='X')

In [ ]:
# Generate cols list with the column names from all datasets
cols = hefzi_df.columns.to_list()+fouladiha_df.columns.to_list()+iCHO2291.columns.to_list()+camel_df.columns.to_list()

# Eliminate repetitive values in the 'cols' list
cols = [cols[i] for i in range(len(cols)) if i == cols.index(cols[i])]

In [ ]:
def add_col(df):
    '''
    This function adds the columns from the cols list 
    that are not present in the df
    '''
    df.columns
    add_col = []
    for col in cols:
        if col not in df.columns:
            add_col.append(col)
    df = df.reindex(columns = df.columns.tolist() + add_col)
    return df

In [ ]:
# Unify columns for all datasets
hefzi_df = add_col(hefzi_df)
fouladiha_df = add_col(fouladiha_df)
iCHO2291 = add_col(iCHO2291)
camel_df = add_col(camel_df)

In [ ]:
# Reorder columns in all datasets the same way
fouladiha_df = fouladiha_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
iCHO2291 = iCHO2291[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
hefzi_df = hefzi_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
camel_df = camel_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]

camel_df['Reaction'] = camel_df['Reaction'].str.strip()

In [ ]:
# Merge all the dfs into a unified df
all_dfs = pd.concat([camel_df, hefzi_df, fouladiha_df, iCHO2291])
all_dfs = all_dfs.reset_index(drop = True)

#Unify reaction names
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace('_cho', '')
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace(r"(e)", "_e_", regex = False)
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace("[", "_")
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace("]", "_")

all_dfs #20940 rows/reactions (many of them repeated)

In [ ]:
# Remove the underscore at the end of some reactions

rxns = []
idx = []

for i,row in all_dfs.iterrows():
    if str(row['Reaction']).endswith('_'):
        s = re.sub('_$', '', row['Reaction'])
        rxns.append(s)
        idx.append(i)
        
all_dfs['Reaction'].update(pd.Series(rxns,index=idx))

### 1.3 Group all the data into a unified dataset
The combined dataset generated above "all_dfs" is grouped by the Reaction BiGG ID to obtain a dataset with unique reaction identifiers

In [ ]:
# Group the data into a unified dataset

all_dfs2 = all_dfs.groupby('Reaction').first()
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace('[','_')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(']','')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(' => | =>',' --> ')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(' <-- | <--',' <=> ')
all_dfs2

In [ ]:
# Separate Demand Reaccion into a different df and remove from reconstruction. 
# Keep demand reactions from extracellular space

all_dfs2 = all_dfs2.reset_index()
demand_reactions = pd.DataFrame(columns = all_dfs2.columns)

for index, rxn in all_dfs2.iterrows():
    rxn = rxn['Reaction Formula']
    a,b = re.split('<=>|-->',rxn)
    if (b == '' or b == ' ') and not a.endswith('_e '):
        temp_df = all_dfs2[all_dfs2['Reaction Formula'] == rxn]
        demand_reactions = pd.concat([demand_reactions,temp_df])
        all_dfs2.drop(index, inplace=True)

demand_reactions = demand_reactions.reset_index(drop=True)
demand_reactions.to_excel('../Data/Reconciliation/datasets/demand_reactions.xlsx')
demand_reactions

In [ ]:
all_dfs2 = all_dfs2.reset_index(drop=True)
all_dfs2

### 1.4 Addition of the Recon3D CHO ortholog GPRs into the reconstruction
The information from the dataset generated in the notebook "GPR Annotation" containing all the information from Recon3D GPRS in human and its corresponding CHO orthologs is mapped into our reconstruction

In [ ]:
# Generate a dict with recon3d reactions as keys and the CHO GPRs as values.
recon3d = pd.read_excel('../Data/GPR_Curation/recon3D_chogprs.xlsx')
recon3d_dict = recon3d.set_index('m_reaction')['CHO GPR'].to_dict()
recon3d_dict

In [ ]:
# Map 'recon3d_dict' into the all_dfs3 dataframe
# the reaction IDs should be the same as those in our reconstruction

all_dfs3 = all_dfs2.reset_index()
all_dfs3['GPR_Recon3D'] = all_dfs3['Reaction'].map(recon3d_dict)

### 1.5 Divide the dataset in two to facilitate reading and curation in Google Sheets
The dataset all_dfs3 is divided into two different datasets: all_dfs4 contains mainly all the information regarding GPRs assigned from previous reconstructions. all_dfs5 contains the rest of the attributes in the reconstruction such as EC number, bounds, etc.

In [ ]:
# all_dfs4 contains mainly information of the reactions GPRs
all_dfs4 = all_dfs3[['Reaction', 'Reaction Name', 'Reaction Formula', 'Subsystem', 'GPR_hef', 'GPR_fou', 'GPR_yeo', 'GPR_Recon3D', 'Curation Notes', 'References']]
all_dfs4.insert(8,'GPR_final', '')
all_dfs4.to_excel('../Data/Reconciliation/datasets/all_dfs4.xlsx')
all_dfs4

In [ ]:
# all_dfs5 contains information of the rest of the attributes in our reconstruction
all_dfs5 = all_dfs3[['Reaction', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible', 'Lower bound', 'Upper bound', 'Objective']]
all_dfs5.to_excel('../Data/Reconciliation/datasets/all_dfs5.xlsx')
all_dfs5

## 2. Identifications of Duplicated Reactions <a id='duplicated'></a>
In this part of the notebook we use the **duplicated_reactions** fucntion from the **utils** module to spot duplicated reactions in our dataset. First we generate a model, using cobprapy, with our dataset, then we apply the **duplicated_reactions** function to the model, and finally we standarize the name of the duplicated reactions according to the nomenclature used in BiGG.

In [ ]:
# Libraries import
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from tqdm.notebook import tqdm
from cobra import Model, Reaction, Metabolite

from utils import duplicated_reactions
from google_sheet import GoogleSheet

### 2.1 Generate the model and identify duplicated reactions
Generation of a cobra model from the **Google Sheet dataset**. This model will be used to identify duplicated reactions from the stoichiometric matrix of the model.

In [ ]:
# Generate the necessary datasets for the identification of the duplicated reactions
KEY_FILE_PATH = 'credentials.json'
SPREADSHEET_ID_v3 = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID_v3, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_rxns = 'Rxns'
sheet_attributes = 'Attributes'

reactions = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)

In [ ]:
##### ----- Create a model from the reactions sheet ----- #####
model = Model("iCHO")
lr = []
for _, row in reactions.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)    
model.add_reactions(lr)
model

In [ ]:
##### ----- Add information to each one of the reactions ----- #####
for i,r in enumerate(tqdm(model.reactions)):
    print(r.id)
    r.build_reaction_from_string(reactions['Reaction Formula'][i])
    r.name = reactions['Reaction Name'][i]
    r.subsystem = reactions['Subsystem'][i]
    r.lower_bound = float(rxns_attributes['Lower bound'][i])
    r.upper_bound = float(rxns_attributes['Upper bound'][i])

In [ ]:
duplicated_reactions = duplicated_reactions(model)
duplicated_reactions

### 2.2 Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above
Duplicated reactions are itereated in a for loop and mapped in the original dataset. A request is made in the 
BiGG database http://bigg.ucsd.edu with each of the duplicated reactions. If any of the duplicated reactions is in BiGG, the other reaction automatically changes its name to the one located in Bigg. This way we unifiy the names of our reactions to those in BiGG.

In [ ]:
i = 0
for rxn in tqdm(duplicated_reactions):
    
    session = HTMLSession()
    
    response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+reactions['Reaction'][rxn[0]])
    if response.status_code == 200:
        if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
            print(f'1 Reaction {reactions.iloc[rxn[1],2]} changed for {reactions.iloc[rxn[0],2]} present in CHOv1 model')
            reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
            rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
            i += 1
    
    elif response.status_code != 200:
        response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+reactions['Reaction'][rxn[1]])
        if response.status_code == 200:
            if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                print(f'2 Reaction {reactions.iloc[rxn[0],2]} changed for {reactions.iloc[rxn[1],2]} present in CHOv1 model')
                reactions.loc[rxn[0], 'Reaction'] = reactions['Reaction'][rxn[1]]
                rxns_attributes.loc[rxn[0], 'Reaction'] = rxns_attributes['Reaction'][rxn[1]]
                i += 1
            
        elif response.status_code != 200:
            response=session.get('http://bigg.ucsd.edu/universal/reactions/'+reactions['Reaction'][rxn[0]])
            if response.status_code == 200:
                if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                    print(f'3 Reaction {reactions.iloc[rxn[1],2]} changed for {reactions.iloc[rxn[0],2]} present BiGG database')
                    reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
                    rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
                    i += 1
                
            elif response.status_code != 200:
                response=session.get('http://bigg.ucsd.edu/universal/reactions/'+reactions['Reaction'][rxn[1]])
                if response.status_code == 200:
                    if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                        print(f'4 Reaction {reactions.iloc[rxn[0],2]} changed for {reactions.iloc[rxn[1],2]} present BiGG database')
                        reactions.loc[rxn[0], 'Reaction'] = reactions['Reaction'][rxn[1]]
                        rxns_attributes.loc[rxn[0], 'Reaction'] = rxns_attributes['Reaction'][rxn[1]]
                        i += 1
                elif response.status_code != 200:
                    if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                        print(f'5 Reaction {reactions.iloc[rxn[1],2]} changed for {reactions.iloc[rxn[0],2]} not present in Bigg DB')
                        reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
                        rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
                        i += 1

print(len(duplicated_reactions))
print(i)

In [ ]:
# Store the original column order
column_order_rxns = reactions.columns.tolist()
column_order_att = rxns_attributes.columns.tolist()

# Group by 'BiGG ID' and keep the first non-null value in each group, then reset the index
reactions = reactions.groupby('Reaction').first().reset_index()
rxns_attributes = rxns_attributes.groupby('Reaction').first().reset_index()

# Rearrange the columns to the original order
reactions = reactions[column_order_rxns]
rxns_attributes = rxns_attributes[column_order_att]

In [ ]:
##############################################################
#### ---------------------------------------------------- ####
#### ---- Update Rxns and  Attributes Google Sheets ----- ####
#### ---------------------------------------------------- ####
##############################################################
sheet.update_google_sheet(sheet_rxns, reactions)
sheet.update_google_sheet(sheet_attributes, rxns_attributes)
print("Google Sheet updated.")

In [ ]:
# Check that Reactions IDs and formulas are equal in "Rxns" and "Attributes" sheets
reactions = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)

rxnsIDseq = list(reactions['Reaction']) == list(rxns_attributes['Reaction'])
if rxnsIDseq:
    print('Reaction IDs in the Rxns and Attributes Sheets are equal\n')
else:
    rxns_sheet_ids = set(list(reactions['Reaction']))
    attr_sheet_ids = set(list(rxns_attributes['Reaction']))
    print(f'Reaction IDs that are in Rxns Sheet and not in Attributes Sheet {rxns_sheet_forms - attr_sheet_forms}\n')
    print(f'Reaction IDs that are in Attributes Sheet and not in Rxns Sheet {attr_sheet_forms - rxns_sheet_forms}\n')

rxnsforseq = list(reactions['Reaction Formula']) == list(rxns_attributes['Reaction Formula'])
if rxnsforseq:
    print('Reaction Formulas in the Rxns and Attributes Sheets are equal')
else:
    rxns_sheet_forms = set(list(reactions['Reaction Formula']))
    attr_sheet_forms = set(list(rxns_attributes['Reaction Formula']))
    print(f'Reaction formulas that are in Rxns Sheet and not in Attributes Sheet {rxns_sheet_forms - attr_sheet_forms}\n')
    print(f'Reaction formulas that are in Attributes Sheet and not in Rxns Sheet {attr_sheet_forms - rxns_sheet_forms}\n')

## 3. Add BiGG database links to every reaction <a id='bigg'></a>
Here we identify which reactions from our dataset are located in BiGG and we add a direct link to that reaction in our dataset

In [1]:
# Libraries import
from requests_html import HTMLSession

from utils import duplicated_reactions
from google_sheet import GoogleSheet

In [2]:
# Generate the necessary datasets for the identification of the duplicated reactions
KEY_FILE_PATH = 'credentials.json'
SPREADSHEET_ID_v3 = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID_v3, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_rxns = 'Rxns'
reactions = sheet.read_google_sheet(sheet_rxns)

In [3]:
reactions

,Package,Curated,Reaction,Reaction Name,Reaction Formula,Subsystem,GPR_hef,GPR_fou,GPR_yeo,GPR_Recon3D,GPR_final,Conf. Score,Curation Notes,References,BiGG database
0,7,PD,10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal",10fthf5glu_c --> 10fthf5glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,,.
1,7,PD,10FTHF5GLUtm,"5-glutamyl-10FTHF transport, mitochondrial",10fthf5glu_m --> 10fthf5glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,None,None
2,7,PD,10FTHF6GLUtl,"6-glutamyl-10FTHF transport, lysosomal",10fthf6glu_c --> 10fthf6glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,None,None
3,7,PD,10FTHF6GLUtm,"6-glutamyl-10FTHF transport, mitochondrial",10fthf6glu_m --> 10fthf6glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,None,None
4,7,PD,10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal",10fthf7glu_c --> 10fthf7glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10501,22,,r2534,Major Facilitator(MFS) TCDB:2.A.1.44.1,thr_L_e <=> thr_L_c,"TRANSPORT, EXTRACELLULAR",,,100757617,100757617,100757617,None,None,None,None
10502,22,,r2535,Major Facilitator(MFS) TCDB:2.A.1.44.1,hom_L_e <=> hom_L_c,TRANSPORT,,,100757617,100757617,100757617,None,None,None,None
10503,22,,r2537,Utilized transport,lnlncgcoa_c <=> lnlncgcoa_r,TRANSPORT,None,None,None,None,None,None,None,None,None
10504,22,,r2538,Utilized transport,dlnlcgcoa_c <=> dlnlcgcoa_r,TRANSPORT,None,None,None,None,None,None,None,None,None


In [10]:
in_big=0
not_bigg=0
for i,row in reactions.iterrows():
    rxn=row['Reaction']
    session = HTMLSession()
    response=session.get('http://bigg.ucsd.edu/universal/reactions/'+rxn)
    if response.status_code == 200:
        print(rxn,'in Bigg')
        reactions.loc[i, 'BiGG database'] = 'http://bigg.ucsd.edu/universal/reactions/'+rxn
        in_big+=1
    if response.status_code != 200:
        print(rxn,'Not in Bigg')
        not_bigg+=1
    
print(in_big)
print(not_bigg)

10FTHF5GLUtl in Bigg
10FTHF5GLUtm in Bigg
10FTHF6GLUtl in Bigg
10FTHF6GLUtm in Bigg
10FTHF7GLUtl in Bigg
10FTHF7GLUtm in Bigg
10FTHFtl in Bigg
10FTHFtm in Bigg
11DOCRTSLte in Bigg
11DOCRTSLtm in Bigg
11DOCRTSLtr in Bigg
11DOCRTSTRNte in Bigg
11DOCRTSTRNtm in Bigg
11DOCRTSTRNtr in Bigg
12DHCHOLabc in Bigg
12DHCHOLt in Bigg
12DHCHOLt2 in Bigg
12HPETATP in Bigg
12HPETUPKt in Bigg
13DAMPPOX in Bigg
13DAMPPte in Bigg
15HPETATP in Bigg
15HPETUPKt in Bigg
15KPROSTGF2c in Bigg
15KPROSTGF2t in Bigg
17AHPRGNLONEte in Bigg
17AHPRGSTRNte in Bigg
1A2425THVITD2t Not in Bigg
1A2425THVITD2tm Not in Bigg
1A25DHVITD3te in Bigg
1MNCAMti in Bigg
1PPDCRp in Bigg
1a25DHVITD3TRn in Bigg
1a_24_25VITD2Hm in Bigg
1a_24_25VITD3Hm in Bigg
1a_25VITD2Hm in Bigg
1a_25VITD3Hm in Bigg
21HPRGNLONE in Bigg
23PDE7pl Not in Bigg
24NPHte in Bigg
24_25DHVITD2t in Bigg
24_25DHVITD2tm in Bigg
24_25DHVITD3t in Bigg
24_25DHVITD3tm in Bigg
24_25VITD2Hm in Bigg
24_25VITD3Hm in Bigg
25HVITD2t in Bigg
25HVITD2tm in Bigg
25HVITD3c i

ACACT184n3m Not in Bigg
ACACT184n3p Not in Bigg
ACACT1r in Bigg
ACACT1rm in Bigg
ACACT1x in Bigg
ACACT201n9p Not in Bigg
ACACT204n3m Not in Bigg
ACACT204n3p Not in Bigg
ACACT204n6m Not in Bigg
ACACT204n6p Not in Bigg
ACACT205n3m Not in Bigg
ACACT205n3p Not in Bigg
ACACT221n9p Not in Bigg
ACACT224n6m Not in Bigg
ACACT224n6p Not in Bigg
ACACT2251n3m Not in Bigg
ACACT2251n3p Not in Bigg
ACACT226n3m Not in Bigg
ACACT226n3p Not in Bigg
ACACT241n9p Not in Bigg
ACACT244n6m Not in Bigg
ACACT244n6p Not in Bigg
ACACT245n6m Not in Bigg
ACACT245n6p Not in Bigg
ACACT246n3p Not in Bigg
ACACT260p Not in Bigg
ACACT4p in Bigg
ACACT50m Not in Bigg
ACACT5p in Bigg
ACACT6p in Bigg
ACACT70m Not in Bigg
ACACT7p in Bigg
ACACT81n3m Not in Bigg
ACACT81n3p Not in Bigg
ACACT8p in Bigg
ACACT90m Not in Bigg
ACACT9p in Bigg
ACACTDMNm Not in Bigg
ACACTMPm Not in Bigg
ACACTTMTDp Not in Bigg
ACACt2 in Bigg
ACACt2m in Bigg
ACACtx in Bigg
ACALDt in Bigg
ACALDtm in Bigg
ACALDtr in Bigg
ACALDtx in Bigg
ACCOAC in Bigg
ACCO

AMPTASECG in Bigg
AMPTASECGe in Bigg
AMPt4 Not in Bigg
AMPtp in Bigg
AMPtr in Bigg
AMY1e in Bigg
AMY2e in Bigg
AND19ONEc in Bigg
ANDRSTANDRte in Bigg
ANDRSTNDNte in Bigg
ANDRSTRNGLCte in Bigg
ANDRSTRNGLCtr in Bigg
ANDRSTRNte in Bigg
ANDRSTRNtr in Bigg
ANTHte in Bigg
ANTIPYRENEte in Bigg
AOBUTDsm in Bigg
AP4AH1 in Bigg
APAT2rm in Bigg
APNNOXte in Bigg
APOCF in Bigg
APOCFm in Bigg
APOC_LYS_BTNP in Bigg
APOC_LYS_BTNPm in Bigg
APPNNte in Bigg
APRGSTRNt1e in Bigg
APRGSTRNte in Bigg
APRTO2 in Bigg
AQCOBALt in Bigg
ARABR in Bigg
ARAB_Lt in Bigg
ARACHCOAtcx Not in Bigg
ARACHCOAtr Not in Bigg
ARACHCOAtx in Bigg
ARACHCPT1 in Bigg
ARACHCPT2 in Bigg
ARACHCRNt in Bigg
ARACHDCOAtr Not in Bigg
ARACHDCOAtx in Bigg
ARACHDFATPtc in Bigg
ARACHDt2 in Bigg
ARACHDtr in Bigg
ARACHETH in Bigg
ARACHFATPc in Bigg
ARACHt in Bigg
ARD in Bigg
ARD1 in Bigg
ARGALAALAt in Bigg
ARGALAPHEt in Bigg
ARGALATHRt in Bigg
ARGARGLYSt in Bigg
ARGARGMETt in Bigg
ARGARGt in Bigg
ARGATB0tc in Bigg
ARGB0AT3tc in Bigg
ARGCYSGLYt in

CDCA3Gte in Bigg
CDCA6AH in Bigg
CDIPTr in Bigg
CDPDAGtm in Bigg
CDPRBTLtg Not in Bigg
CDPtg Not in Bigg
CDS in Bigg
CDSm in Bigg
CE1273t1 in Bigg
CE1352te in Bigg
CE1447te in Bigg
CE1918te in Bigg
CE2006te in Bigg
CE2026te in Bigg
CE2209te in Bigg
CE2211te in Bigg
CE2510ABCt in Bigg
CE2537ABCt in Bigg
CE2705t in Bigg
CE4877te in Bigg
CE4890te in Bigg
CE4890te2 in Bigg
CE4968te in Bigg
CE4970te in Bigg
CE5025t in Bigg
CE5026t in Bigg
CE5072te in Bigg
CE6205te in Bigg
CE6247t in Bigg
CE7085te in Bigg
CE7090te in Bigg
CE7096te in Bigg
CEPTC in Bigg
CEPTE in Bigg
CERK in Bigg
CERT1gt in Bigg
CERT1rt in Bigg
CERT2gt in Bigg
CERT2rt in Bigg
CGLYt3_2 in Bigg
CGMPt in Bigg
CH25H in Bigg
CHAT in Bigg
CHATn in Bigg
CHLP in Bigg
CHLPCTD in Bigg
CHLtm in Bigg
CHOLATEt in Bigg
CHOLATEt2 in Bigg
CHOLATEt3 in Bigg
CHOLD2m in Bigg
CHOLESACATc in Bigg
CHOLESTTDe in Bigg
CHOLESTle in Bigg
CHOLK in Bigg
CHOLPtg in Bigg
CHOLPtl in Bigg
CHOLt4 in Bigg
CHOLtg in Bigg
CHOLtn in Bigg
CHOLtr in Bigg
CHOLtu in

DIGALSGALSIDEtg in Bigg
DIGALSIDEtg in Bigg
DIGALSIDEtl in Bigg
DIHOLINETH in Bigg
DINt in Bigg
DITHYRSULFO Not in Bigg
DITPtn in Bigg
DKMPPD2 in Bigg
DLCLAT Not in Bigg
DLNLCGCOAtcx Not in Bigg
DLNLCGCPT1 in Bigg
DLNLCGCPT2 in Bigg
DLNLCGCRNt in Bigg
DLNLCGt in Bigg
DMANTIPYRINEte in Bigg
DMARG in Bigg
DMATT in Bigg
DMATTx in Bigg
DMBADLMO Not in Bigg
DMBADLMOr Not in Bigg
DMBAMO Not in Bigg
DMBAMOr Not in Bigg
DMGDHm in Bigg
DMGtm in Bigg
DMHPTCRNCPT1 in Bigg
DMHPTCRNCPT2 in Bigg
DMHPTCRNt in Bigg
DMHPTCRNte in Bigg
DMNONCOACRNCPT1 in Bigg
DMNONCOAtcx Not in Bigg
DMNONCRNCPT2 in Bigg
DMNONCRNt in Bigg
DMT in Bigg
DNADDP in Bigg
DNADtn in Bigg
DNAMTSEn in Bigg
DNAMTn in Bigg
DNAsyn Not in Bigg
DNAsyn_prod Not in Bigg
DNDPt10m in Bigg
DNDPt11m in Bigg
DNDPt12m in Bigg
DNDPt13m in Bigg
DNDPt14m in Bigg
DNDPt15m in Bigg
DNDPt16m in Bigg
DNDPt17m in Bigg
DNDPt18m in Bigg
DNDPt19m in Bigg
DNDPt1m in Bigg
DNDPt20m in Bigg
DNDPt21m in Bigg
DNDPt22m in Bigg
DNDPt23m in Bigg
DNDPt24m in Bigg
D

...

PEDH161_HSPLA2 in Bigg
PEDH17_HSPLA2 in Bigg
PEDH203_HSABCt in Bigg
PEDH203_HSPLA2 in Bigg
PEFLIP in Bigg
PEFLIPm in Bigg
PEHSFABPe in Bigg
PELINETH in Bigg
PELINL_HSABCt in Bigg
PELINL_HSPLA2 in Bigg
PELPALM_HSABCt in Bigg
PENDECAETH in Bigg
PEOLE_HSABCt in Bigg
PEOLE_HSPLA2 in Bigg
PEPACONtm Not in Bigg
PEPALM in Bigg
PEPALM_HSASBCt in Bigg
PEPALM_HSPLA2 in Bigg
PEPCK in Bigg
PEPCKm in Bigg
PEPLYStn in Bigg
PERILLYLte in Bigg
PEROXx in Bigg
PESTE_HSABCt in Bigg
PETHCT in Bigg
PETOHMm_hs in Bigg
PETOHMr_hs in Bigg
PE_HStg in Bigg
PE_HStm in Bigg
PEt in Bigg
PFK in Bigg
PFK26 in Bigg
PGAM1_PhosHydro in Bigg
PGCD in Bigg
PGDI in Bigg
PGDIr in Bigg
PGESc in Bigg
PGESr in Bigg
PGI in Bigg
PGISr in Bigg
PGK in Bigg
PGL in Bigg
PGLYCABCte in Bigg
PGLYCH Not in Bigg
PGLYCP in Bigg
PGLYCt in Bigg
PGLer in Bigg
PGM in Bigg
PGMT in Bigg
PGPPT in Bigg
PGPP_hs in Bigg
PGS in Bigg
PGSr in Bigg
PHACCOAGLNAC in Bigg
PHACCOAGLYACm in Bigg
PHCDm in Bigg
PHCHGSm in Bigg
PHEACGLNt in Bigg
PHEACGLYte Not

RE0925R in Bigg
RE0926C in Bigg
RE0926E in Bigg
RE0927C Not in Bigg
RE0927R in Bigg
RE0928C Not in Bigg
RE0928R in Bigg
RE0935C in Bigg
RE0935E in Bigg
RE0936C in Bigg
RE0936E in Bigg
RE0937C in Bigg
RE0937E in Bigg
RE0938C in Bigg
RE0938E in Bigg
RE0944C in Bigg
RE0944E in Bigg
RE0958C in Bigg
RE0958E in Bigg
RE1050C Not in Bigg
RE1050E in Bigg
RE1050L Not in Bigg
RE1050N Not in Bigg
RE1062C in Bigg
RE1062M in Bigg
RE1063C in Bigg
RE1064C in Bigg
RE1077C in Bigg
RE1096C in Bigg
RE1096M in Bigg
RE1099C in Bigg
RE1099G Not in Bigg
RE1099L Not in Bigg
RE1099R Not in Bigg
RE1100G Not in Bigg
RE1100L Not in Bigg
RE1134C in Bigg
RE1134M in Bigg
RE1135G Not in Bigg
RE1135L Not in Bigg
RE1233C in Bigg
RE1233M Not in Bigg
RE1234C in Bigg
RE1235C in Bigg
RE1236C in Bigg
RE1238X in Bigg
RE1240C in Bigg
RE1266C in Bigg
RE1308C in Bigg
RE1308M in Bigg
RE1309C in Bigg
RE1309M in Bigg
RE1310C in Bigg
RE1310M in Bigg
RE1311C in Bigg
RE1311M in Bigg
RE1317C in Bigg
RE1342C in Bigg
RE1441G in Bigg
RE14

RE2914X Not in Bigg
RE2915M Not in Bigg
RE2915X Not in Bigg
RE2916M Not in Bigg
RE2916X Not in Bigg
RE2917X Not in Bigg
RE2919M Not in Bigg
RE2919X Not in Bigg
RE2920X Not in Bigg
RE2921M Not in Bigg
RE2921X Not in Bigg
RE2948C in Bigg
RE2954C in Bigg
RE2958C in Bigg
RE2972G in Bigg
RE2972M in Bigg
RE2972R in Bigg
RE2973G in Bigg
RE2973N in Bigg
RE2973R in Bigg
RE2974G in Bigg
RE2974N in Bigg
RE2974R in Bigg
RE2975C in Bigg
RE2975M in Bigg
RE2977C Not in Bigg
RE2985M in Bigg
RE2985X in Bigg
RE2986X in Bigg
RE2987X in Bigg
RE2988X in Bigg
RE2989X in Bigg
RE2990X in Bigg
RE2991X in Bigg
RE2992M in Bigg
RE2992X in Bigg
RE2993X in Bigg
RE2994X in Bigg
RE2995M Not in Bigg
RE2995X Not in Bigg
RE2996X in Bigg
RE2997X in Bigg
RE2998M in Bigg
RE2999M in Bigg
RE3000M in Bigg
RE3001M in Bigg
RE3002X in Bigg
RE3003M in Bigg
RE3004M in Bigg
RE3005M in Bigg
RE3006M in Bigg
RE3009C in Bigg
RE3010C in Bigg
RE3010M in Bigg
RE3010R in Bigg
RE3010X in Bigg
RE3011M in Bigg
RE3011R in Bigg
RE3012C in Bigg


RE3525C in Bigg
RE3525M in Bigg
RE3525N in Bigg
RE3525R in Bigg
RE3525X in Bigg
RE3526C in Bigg
RE3526M in Bigg
RE3526X in Bigg
RE3532C in Bigg
RE3532M in Bigg
RE3532R in Bigg
RE3533C in Bigg
RE3533M in Bigg
RE3533R in Bigg
RE3534C in Bigg
RE3534M in Bigg
RE3534R in Bigg
RE3535R in Bigg
RE3536C in Bigg
RE3537C in Bigg
RE3550X in Bigg
RE3551X in Bigg
RE3552X in Bigg
RE3554C in Bigg
RE3554M in Bigg
RE3554R in Bigg
RE3556C in Bigg
RE3557C in Bigg
RE3557M in Bigg
RE3557R in Bigg
RE3559M in Bigg
RE3559X in Bigg
RE3560C in Bigg
RE3560M in Bigg
RE3560X in Bigg
RE3561M in Bigg
RE3561X in Bigg
RE3562C in Bigg
RE3562M in Bigg
RE3562R in Bigg
RE3562X in Bigg
RE3563M in Bigg
RE3563X in Bigg
RE3564C in Bigg
RE3564M in Bigg
RE3564X in Bigg
RE3565C in Bigg
RE3566C in Bigg
RE3567C in Bigg
RE3568C in Bigg
RE3570C in Bigg
RE3571C in Bigg
RE3571R in Bigg
RE3572X in Bigg
RE3573X in Bigg
RE3574X in Bigg
RE3575X in Bigg
RE3576X in Bigg
RE3577X in Bigg
RE3578X in Bigg
RE3580X in Bigg
RE3581X in Bigg
RE3582X 

KeyboardInterrupt: 

In [12]:
print(in_big)
print(not_bigg)

6432
1693


In [11]:
reactions

,Package,Curated,Reaction,Reaction Name,Reaction Formula,Subsystem,GPR_hef,GPR_fou,GPR_yeo,GPR_Recon3D,GPR_final,Conf. Score,Curation Notes,References,BiGG database
0,7,PD,10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal",10fthf5glu_c --> 10fthf5glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,,http://bigg.ucsd.edu/universal/reactions/10FTH...
1,7,PD,10FTHF5GLUtm,"5-glutamyl-10FTHF transport, mitochondrial",10fthf5glu_m --> 10fthf5glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,None,http://bigg.ucsd.edu/universal/reactions/10FTH...
2,7,PD,10FTHF6GLUtl,"6-glutamyl-10FTHF transport, lysosomal",10fthf6glu_c --> 10fthf6glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,None,http://bigg.ucsd.edu/universal/reactions/10FTH...
3,7,PD,10FTHF6GLUtm,"6-glutamyl-10FTHF transport, mitochondrial",10fthf6glu_m --> 10fthf6glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,None,http://bigg.ucsd.edu/universal/reactions/10FTH...
4,7,PD,10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal",10fthf7glu_c --> 10fthf7glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,None,http://bigg.ucsd.edu/universal/reactions/10FTH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10501,22,,r2534,Major Facilitator(MFS) TCDB:2.A.1.44.1,thr_L_e <=> thr_L_c,"TRANSPORT, EXTRACELLULAR",,,100757617,100757617,100757617,None,None,None,http://bigg.ucsd.edu/models/iCHOv1/reactions/r...
10502,22,,r2535,Major Facilitator(MFS) TCDB:2.A.1.44.1,hom_L_e <=> hom_L_c,TRANSPORT,,,100757617,100757617,100757617,None,None,None,http://bigg.ucsd.edu/models/iCHOv1/reactions/r...
10503,22,,r2537,Utilized transport,lnlncgcoa_c <=> lnlncgcoa_r,TRANSPORT,None,None,None,None,None,None,None,None,http://bigg.ucsd.edu/models/iCHOv1/reactions/r...
10504,22,,r2538,Utilized transport,dlnlcgcoa_c <=> dlnlcgcoa_r,TRANSPORT,None,None,None,None,None,None,None,None,http://bigg.ucsd.edu/models/iCHOv1/reactions/r...


In [9]:
#############################################
#### ----------------------------------- ####
#### ---- Update Rxns Google Sheet ----- ####
#### ----------------------------------- ####
#############################################
sheet.update_google_sheet(sheet_rxns, reactions)
print("Google Sheet updated.")

Google Sheet updated.


## 4. Divide into compartments <a id='compartments'></a>

In [ ]:
from cobra.io import load_json_model
import pandas as pd

# model = read_sbml_model('iCHOv3_CHO_05072023.xml')
model = load_json_model('iCHOv3_CHO_05072023.json')

In [ ]:
subsystem_list = []
for r in model.reactions:
    subsystem_list.append(r.subsystem)
print("Number of unique subsystems:", len(set(subsystem_list)))

In [ ]:
subsystem_counts = pd.Series(subsystem_list).value_counts().reset_index()
subsystem_counts.columns = ['Subsystem', 'RXN Counts']
subsystem_counts[subsystem_counts['RXN Counts'] == 1]


In [ ]:
subsystem_counts

In [ ]:
# TODO -Remove blocked reactions from packages
#      -Define the high-importance reactions for biomass production
#      -Rename subsystems with very few reactions